In [1]:
# Because MobileNet has version that is based out of TensorFlow 1
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)


!git clone --quiet https://github.com/tensorflow/models.git

TensorFlow 1.x selected.
1.15.2


In [ ]:
# Install the TF Object Detection API (TFOD)
%%bash
pip install --upgrade pip
cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf1/setup.py .
python -m pip install --use-feature=2020-resolver .

In [27]:
from subprocess import check_call

i = 1

checkpoint_links = {
    'ssd_mobilenet_v2_coco_2018_03_29': 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz',
    'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03': 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz',
    'ssdlite_mobilenet_v2_coco_2018_05_09': 'http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz',
    'ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18': 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18.tar.gz',
    'ssd_mobilenet_v3_large_coco_2020_01_14': 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz',
    'ssd_mobilenet_v3_small_coco_2020_01_14': 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz'
}

folder_name = [a for a in checkpoint_links.keys()][i]
checkpoint_link = [a for a in checkpoint_links.values()][i]

print(folder_name)
print(checkpoint_link)

!rm -rf folder_name
!wget -q $checkpoint_link -O checkpoints.tar.gz
!tar -xvf checkpoints.tar.gz
!rm checkpoints.tar.gz

ssd_mobilenet_v2_coco_2018_03_29
http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
ssd_mobilenet_v2_coco_2018_03_29/checkpoint
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.meta
ssd_mobilenet_v2_coco_2018_03_29/pipeline.config
ssd_mobilenet_v2_coco_2018_03_29/saved_model/saved_model.pb
ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb
ssd_mobilenet_v2_coco_2018_03_29/saved_model/
ssd_mobilenet_v2_coco_2018_03_29/saved_model/variables/
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.index
ssd_mobilenet_v2_coco_2018_03_29/
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.data-00000-of-00001


In [ ]:
#Export a TFLite compatible model graph
i = 0

# For simple mobilenetV2 model https://stackoverflow.com/questions/49880939/tf-object-detection-api-detection-model-retraining-object-detection-protos-ssd/49883742#49883742


folder_name = [a for a in checkpoint_links.keys()][i]

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path=$folder_name/pipeline.config \
    --trained_checkpoint_prefix=$folder_name/model.ckpt \
    --output_directory=$folder_name \
    --add_postprocessing_op=true
print("======Graph generated========")
!ls -lh $folder_name/*.pb

In [1]:
#Initial setup
%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

'2.3.0'

In [2]:
#Select model to quantize
import os

model_checkpoint_name = "ssd_mobilenet_v2_quantized_coco_fp32"
model_to_be_quantized = 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/tflite_graph.pb'
if os.path.getsize(model_to_be_quantized) > 0:
    print(f"We are quantizing: {model_checkpoint_name}")
else:
    print("Model not found!")

We are quantizing: ssd_mobilenet_v2_quantized_coco_fp32


In [ ]:
# Representative dataset:
!wget -q https://github.com/sayakpaul/Adventures-in-TensorFlow-Lite/releases/download/v0.9.0/train_samples_coco.tar.gz
!tar -xvf train_samples_coco.tar.gz
!rm train_samples_coco.tar.gz

rep_ds = tf.data.Dataset.list_files("train_samples/*.jpg")
HEIGHT, WIDTH = 300, 300

def representative_dataset_gen():
    for image_path in rep_ds:
        img = tf.io.read_file(image_path)
        img = tf.io.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        resized_img = tf.image.resize(img, (HEIGHT, WIDTH))
        resized_img = resized_img[tf.newaxis, :]
        yield [resized_img]

In [4]:
#Quantize and serialize (`uint8` are WIP)
original_precision = "fp32"
quantization_strategy = "int8"

converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file=model_to_be_quantized, 
    input_arrays=['normalized_input_image_tensor'],
    output_arrays=['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'],
    input_shapes={'normalized_input_image_tensor': [1, 300, 300, 3]}
)
converter.allow_custom_ops = True

assert model_checkpoint_name.split("_")[-1]==original_precision, "Mismatch between the precisions."
assert quantization_strategy == "int8", "Only integer precision is allowed for this model."
# https://github.com/ml-gde/e2e-tflite-tutorials/issues/21#issuecomment-692063937
converter.inference_input_type = tf.uint8
converter.quantized_input_stats = {"normalized_input_image_tensor": (128, 128)}
# No additional optimization: https://github.com/ml-gde/e2e-tflite-tutorials/issues/21#issuecomment-692545579

tflite_model = converter.convert()

tflite_filename = model_checkpoint_name + "_" + quantization_strategy + ".tflite"
open(tflite_filename, 'wb').write(tflite_model)
print(f"TFLite model generated with {quantization_strategy}")
!ls -lh $tflite_filename

TFLite model generated with int8
-rw-r--r-- 1 root root 6.0M Nov 24 09:13 ssd_mobilenet_v2_quantized_coco_fp32_int8.tflite


In [ ]:
tflite_name = 'ssd_mobilenet_v2_quantized_coco_fp32_int8'

In [ ]:
#Install the latest edge TPU compiler
%%bash
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
sudo apt-get update
sudo apt-get install edgetpu-compiler

In [ ]:
#Compile the file
!edgetpu_compiler -s "$tflite_name".tflite

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 1214 ms.

Input model: ssd_mobilenet_v2_quantized_coco_fp32_int8.tflite
Input size: 5.93MiB
Output model: ssd_mobilenet_v2_quantized_coco_fp32_int8_edgetpu.tflite
Output size: 6.59MiB
On-chip memory used for caching model parameters: 6.35MiB
On-chip memory remaining for caching model parameters: 1.39MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 111
Operation log: ssd_mobilenet_v2_quantized_coco_fp32_int8_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 108
Number of operations that will run on CPU: 3

Operator                       Cou

## Download model

In [ ]:
#Download model
from google.colab import files
files.download(tflite_name + '_edgetpu.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Try all models

In [11]:
checkpoint_links = {
    'ssd_mobilenet_v2_coco_2018_03_29': 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz',
    'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03': 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz',
    'ssdlite_mobilenet_v2_coco_2018_05_09': 'http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz',
    'ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18': 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18.tar.gz',
    'ssd_mobilenet_v3_large_coco_2020_01_14': 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz',
    'ssd_mobilenet_v3_small_coco_2020_01_14': 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz'
}
img_sizes = {
    'ssd_mobilenet_v2_coco_2018_03_29': 300,
    'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03': 300,
    'ssdlite_mobilenet_v2_coco_2018_05_09': 300,
    'ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18': 320,
    'ssd_mobilenet_v3_large_coco_2020_01_14': 320,
    'ssd_mobilenet_v3_small_coco_2020_01_14': 320}

In [9]:
i = 2

folder_name = [a for a in checkpoint_links.keys()][i]
img_size = [a for a in img_sizes.keys()][i]

model_checkpoint_name = folder_name
model_to_be_quantized = folder_name + '/tflite_graph.pb'

converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file=model_to_be_quantized, 
    input_arrays=['normalized_input_image_tensor'],
    output_arrays=['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'],
    input_shapes={'normalized_input_image_tensor': [1, img_size, img_size, 3]}
)
converter.allow_custom_ops = True
tflite_model = converter.convert()

tflite_filename = model_checkpoint_name + ".tflite"
open(tflite_filename, 'wb').write(tflite_model)
print(f"TFLite model generated without conversion.")
!ls -lh $tflite_filename

TFLite model generated without conversion.
-rw-r--r-- 1 root root 6.9M Nov 24 12:07 ssd_mobilenet_v3_small_coco_2020_01_14.tflite


In [10]:
#Download model
from google.colab import files
import glob

for file in glob.glob('./*tflite'):
  files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>